In [17]:
import scraping_class
import requests
import random
import tqdm
import pandas as pd
import time
import numpy as np
import datetime
import pytest
import requests
#from connector import Connector
logfile = 'log.csv'## name your log file.
connector = scraping_class.Connector(logfile) # for logging

from numpy.random import randint
import logging

#
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re
import os
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

In [18]:
# Set up Chrome WebDriver
driver = webdriver.Chrome()
driver.implicitly_wait(10)

# header
headers = {'email': 'qls153@alumni.ku.dk', 'name': 'Nina Frandsen', 'message': 'Students from university of Copenhagen - doing a project on machine learning'}

# Define the search term and category
search_term = 'a'

# Navigate to the search page
url = f'https://ekstrabladet.dk/find/?g=true&q={search_term}'

# Function to check if the article belongs to the desired category
def is_desired_category(article):
    card_meta = article.find(class_='card_meta')
    if card_meta:
        spans = card_meta.find_all('span')
        for span in spans:
            if 'Dansk politik' in span.get_text():
                return True
    return False

driver.get(url)

# Scrape articles
loops = 0
urls = []

while True:
    loops += 1
    print(f'{loops} loops')
    
    # Scroll down to load more articles
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Wait for content to load
    
    # Get the page source after scrolling
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # Save soup
    with open("search_html.txt", "w", encoding="utf-8") as f:
        f.write(soup.prettify())  # Save prettified HTML
        f.close()
    
    # Find articles on the page
    articles = soup.find_all(class_='headline-list__item')

    for article in articles:
        if is_desired_category(article):
            article_url = article.find('a')['href']
            urls.append(article_url)
    
    # Save every 100 loops
    if loops % 100 == 0:
        df_urls = pd.DataFrame(data=urls)
        df_urls.to_csv("urls.csv", index=False)
        print(f'Saved {len(urls)} urls')

    # Check if there's a "Load more" button and break the loop if not
    load_more_button = driver.find_elements(by='xpath', value='//button[contains(text(), "Hent flere")]')
    if not load_more_button:
        print('No more articles to load')
        break
    else:
        load_more_button[0].click()
        time.sleep(2)  # Wait for content to load

# Close the Chrome WebDriver
driver.quit()

1 loops
Saved 0 urls
No more articles to load


In [19]:
# load urls
urls = pd.read_csv("urls.csv")

EmptyDataError: No columns to parse from file

In [ ]:
# remove plus articles
urls_no_plus = [ url for url in urls['0'] if "https://ekstrabladet.dk/plus" not in url ]

In [ ]:
'''
The data is being split up into smaller chunks to scrape in on multiple computers.
this is done in the bots folder.
'''
#split into equal chunks of urls.
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
chunks = list(chunks(urls_no_plus, 8800))
print(f'there are {len(chunks)} chunks')
chunk0 = chunks[0]
chunk1 = chunks[1]
chunk2 = chunks[2]
chunk3 = chunks[3]
# save chunks
chunk0 = pd.DataFrame(data=chunk0)
chunk0.to_csv("chunk0.csv", index=False)
chunk1 = pd.DataFrame(data=chunk1)
chunk1.to_csv("chunk1.csv", index=False)
chunk2 = pd.DataFrame(data=chunk2)
chunk2.to_csv("chunk2.csv", index=False)
chunk3 = pd.DataFrame(data=chunk3)
chunk3.to_csv("chunk3.csv", index=False)

In [ ]:
# load scraped data
df_scraped = pd.read_csv("data.csv")

In [ ]:
'''
The following code takes the scraped headlines, articles, dates etc.
and converts it to a df and the dates to timeseries.
'''

# convert to df and drop nan
df = pd.DataFrame(df_scraped).dropna()
df=df.reset_index(drop=True)

x=len(df)
dato=[]

# convert times 
for i  in range(x):
    data_time_string=df[3][i]
    x=re.sub('\.',"",data_time_string)
    y=datetime.datetime.strptime(x, '%d %b %Y')
    dato.append(y)
    time.sleep(0.5)
    

df['time']=pd.Series(dato).values